# Final Assignment: Analyzing Historical Stock/Revenue Data and Building a Dashboard

This notebook completes the assignment requirements. It downloads/loads stock and revenue data for Tesla (TSLA) and GameStop (GME), performs the requested cleaning and analysis, plots the results, and answers the written questions.

**Notes:**
- If you run this notebook locally or on Colab, the cells that download data from the web will attempt to fetch live data. If your environment restricts internet access, replace the download cells with local CSV files named `tesla_stock.csv`, `tesla_revenue.csv`, `gme_stock.csv`, and `gme_revenue.csv` in the working directory.
- The notebook is fully executed when run; all output cells will display results when executed in order.

---

## 0. Setup and imports


In [ ]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

# Optional libraries for fetching data
try:
    import yfinance as yf
except Exception as e:
    print('yfinance not available. You can install it with `pip install yfinance` if you want to fetch stock data automatically.')

try:
    from bs4 import BeautifulSoup
    import requests
except Exception as e:
    print('BeautifulSoup/requests not available. You can install them with `pip install bs4 requests` if you want to fetch revenue data automatically.')

# Notebook-wide plotting settings
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['tight_layout'] = True


## 1. Download or load the datasets

This notebook tries to fetch stock prices using `yfinance` and revenue data by scraping MacroTrends. If internet or required packages are not available, please place the CSV files (`tesla_stock.csv`, `tesla_revenue.csv`, `gme_stock.csv`, `gme_revenue.csv`) in the working directory and re-run the notebook.

In [ ]:
# File paths
tsla_stock_csv = 'tesla_stock.csv'
tsla_rev_csv   = 'tesla_revenue.csv'
gme_stock_csv  = 'gme_stock.csv'
gme_rev_csv    = 'gme_revenue.csv'

# 1A: Try to download stock data with yfinance (if available)
def fetch_stock(ticker, start='2000-01-01', end=None, csv_path=None):
    if csv_path and os.path.exists(csv_path):
        print(f'Loading {csv_path} from disk.')
        return pd.read_csv(csv_path)
    if 'yf' in globals():
        print(f'Fetching {ticker} from yfinance...')
        data = yf.download(ticker, start=start, end=end)
        data = data.reset_index()
        if csv_path:
            data.to_csv(csv_path, index=False)
        return data
    raise RuntimeError('yfinance not available and CSV not found.')

# 1B: Try to scrape revenue tables from MacroTrends (best-effort)
def fetch_revenue_macrotrends(ticker, csv_path=None):
    # MacroTrends pages (structure may change): example
    url = f'https://www.macrotrends.net/stocks/charts/{ticker}/{ticker}/revenue'
    if csv_path and os.path.exists(csv_path):
        print(f'Loading {csv_path} from disk.')
        return pd.read_csv(csv_path)
    try:
        print(f'Fetching revenue data from {url} ...')
        resp = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
        soup = BeautifulSoup(resp.text, 'html.parser')
        # Find the table that contains 'Monthly' or 'Quarterly' revenue; MacroTrends uses tables with class 'historical_data_table'
        tables = soup.find_all('table')
        # Heuristic: locate table with 'Revenue' header
        for table in tables:
            if 'Revenue' in str(table) or 'Quarter' in str(table):
                df = pd.read_html(str(table))[0]
                # Basic cleaning
                df.columns = [c.strip() for c in df.columns]
                if 'Revenue' in df.columns or 'Total Revenue' in df.columns or df.shape[1] >= 2:
                    df.to_csv(csv_path, index=False) if csv_path else None
                    return df
        raise ValueError('No revenue table found on page (structure may have changed).')
    except Exception as e:
        print('Failed to fetch revenue from MacroTrends:', e)
        raise

# Attempt to load/fetch
try:
    tesla_data = fetch_stock('TSLA', start='2002-01-01', csv_path=tsla_stock_csv)
except Exception as e:
    print('TESLA stock load error:', e)
    tesla_data = pd.DataFrame()

try:
    gme_data = fetch_stock('GME', start='2002-01-01', csv_path=gme_stock_csv)
except Exception as e:
    print('GME stock load error:', e)
    gme_data = pd.DataFrame()

# Revenue fetch attempts (best-effort)
try:
    tesla_revenue = fetch_revenue_macrotrends('TSLA', csv_path=tsla_rev_csv)
except Exception as e:
    print('TESLA revenue load error:', e)
    tesla_revenue = pd.DataFrame()

try:
    gme_revenue = fetch_revenue_macrotrends('GME', csv_path=gme_rev_csv)
except Exception as e:
    print('GME revenue load error:', e)
    gme_revenue = pd.DataFrame()

# Show what we have
print('tesla_data rows:', len(tesla_data))
print('tesla_revenue rows:', len(tesla_revenue))
print('gme_data rows:', len(gme_data))
print('gme_revenue rows:', len(gme_revenue))


## 2. Data cleaning & answering the in-notebook checks (Question 1 subparts)

We will:
- Display first/last rows as requested
- Convert `Date` columns to datetime
- Remove `$` and commas from monetary columns when needed
- Plot stock price and revenue for each company

Each displayed result corresponds to the grading checklist.

In [ ]:
# Helper to ensure Date is datetime
def ensure_date(df, date_col='Date'):
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    return df

# Standardize column names for revenue data if possible
def normalize_revenue_df(df):
    if df.empty:
        return df
    # try to identify date-like and revenue-like columns
    cols = df.columns.tolist()
    # common macrotrends format: ['Year', 'Revenue', '...']
    # find revenue-like column
    revenue_col = None
    for c in cols:
        if 'Revenue' in c or 'Total' in c or '$' in c:
            revenue_col = c
            break
    # find date-like column
    date_col = cols[0] if len(cols) >= 2 else cols[0]
    # rename to Date/Revenue
    new_df = df.rename(columns={date_col: 'Date', revenue_col: 'Revenue'} if revenue_col else {date_col: 'Date'})
    # clean Revenue column
    if 'Revenue' in new_df.columns:
        new_df['Revenue'] = new_df['Revenue'].astype(str).str.replace('\$', '', regex=True).str.replace(',', '', regex=True)
        # convert numeric when possible
        new_df['Revenue'] = pd.to_numeric(new_df['Revenue'].str.replace('\(|\)', '', regex=True), errors='coerce')
    # convert Date
    new_df = ensure_date(new_df, 'Date')
    return new_df

tesla_data = ensure_date(tesla_data, 'Date')
gme_data = ensure_date(gme_data, 'Date')
tesla_revenue = normalize_revenue_df(tesla_revenue)
gme_revenue = normalize_revenue_df(gme_revenue)

# Q1.2: Display first five rows of tesla_data
print('--- Tesla stock (first 5 rows) ---')
display(tesla_data.head(5))

# Q1.3: Display last five rows of tesla_revenue
print('--- Tesla revenue (last 5 rows) ---')
display(tesla_revenue.tail(5))

# Q1.4: First five rows of gme_data
print('--- GME stock (first 5 rows) ---')
display(gme_data.head(5))

# Q1.5: Last five rows of gme_revenue
print('--- GME revenue (last 5 rows) ---')
display(gme_revenue.tail(5))

# Q1.6: Plot both graphs for Tesla (stock and revenue)
if not tesla_data.empty:
    fig, ax1 = plt.subplots()
    ax1.plot(tesla_data['Date'], tesla_data['Close'], label='TSLA Close')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('TSLA Close Price (USD)')
    ax1.legend(loc='upper left')
    plt.title('Tesla Stock Price')
    plt.show()
else:
    print('No Tesla stock data to plot.')

if not tesla_revenue.empty:
    fig, ax = plt.subplots()
    ax.plot(tesla_revenue['Date'], tesla_revenue['Revenue'], marker='o')
    ax.set_xlabel('Date')
    ax.set_ylabel('Revenue (USD)')
    ax.set_title('Tesla Revenue (historical)')
    plt.show()
else:
    print('No Tesla revenue data to plot.')

# Q1.7: Plot both graphs for GameStop
if not gme_data.empty:
    fig, ax1 = plt.subplots()
    ax1.plot(gme_data['Date'], gme_data['Close'], label='GME Close')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('GME Close Price (USD)')
    ax1.legend(loc='upper left')
    plt.title('GameStop Stock Price')
    plt.show()
else:
    print('No GME stock data to plot.')

if not gme_revenue.empty:
    fig, ax = plt.subplots()
    ax.plot(gme_revenue['Date'], gme_revenue['Revenue'], marker='o')
    ax.set_xlabel('Date')
    ax.set_ylabel('Revenue (USD)')
    ax.set_title('GameStop Revenue (historical)')
    plt.show()
else:
    print('No GME revenue data to plot.')


## 3. Question 2 — BeautifulSoup parsers

**Question:** Which parsers can be used in BeautifulSoup to parse HTML data? (2 points)

**Answer:** BeautifulSoup supports several parsers. The most common are:

- `html.parser` (Python’s built-in HTML parser)
- `lxml` (very fast; requires `lxml` library)
- `lxml-xml` (for XML parsing)
- `html5lib` (very permissive, creates a parse tree similar to modern browsers)

You choose a parser by passing the `features` or `parser` argument to `BeautifulSoup`, e.g. `BeautifulSoup(html, 'lxml')`.

## 4. Question 3 — Why convert Date to datetime? (1 point)

**Answer:** Converting the `Date` column to a `datetime` type allows for accurate chronological sorting, time-based indexing, resampling (e.g., to monthly or yearly frequency), date arithmetic, and correct plotting on time axes. It also helps avoid errors when filtering by date ranges.

## 5. Question 4 — Remove `$` and commas from a Price column (2 points)

**Correct option (example code):**

```python
# assuming df['Price'] contains strings like "$1,234.56"
df['Price'] = df['Price'].str.replace('\$', '', regex=True).str.replace(',', '', regex=True).astype(float)
```

This removes the dollar sign and commas, then converts the column to a float.

## 6. Question 5 — What is the 'Revenue' value in the last row of the extracted `tesla_revenue` dataset? (2 points)

Below we print the last row and report the Revenue value. If the dataset is empty (not fetched), the cell will indicate that and you should place your `tesla_revenue.csv` file in the working directory and re-run the notebook.

In [ ]:
if not tesla_revenue.empty:
    last_row = tesla_revenue.dropna(subset=['Revenue']).tail(1)
    print('Last row of tesla_revenue:')
    display(last_row)
    try:
        print('Revenue value (last row):', last_row['Revenue'].iloc[0])
    except Exception as e:
        print('Could not extract Revenue value:', e)
else:
    print('tesla_revenue is empty. Add tesla_revenue.csv or ensure scraping worked and re-run.')

## 7. Question 6 — What is the 'Close' price in the first row (Date: 2002-02-13) of the extracted GameStop stock data? (2 points)

We will attempt to find the first row with that date and print the Close price. If the exact date doesn't exist in the dataset, the notebook will show the available first row.

In [ ]:
query_date = pd.to_datetime('2002-02-13')
if not gme_data.empty:
    # find row with that date (exact) or the first row
    exact = gme_data[gme_data['Date'] == query_date]
    if not exact.empty:
        print('Close price on 2002-02-13:', exact['Close'].iloc[0])
        display(exact)
    else:
        print('No row with date 2002-02-13. Showing the first row instead:')
        display(gme_data.head(1))
        try:
            print('First row Close price:', gme_data['Close'].iloc[0])
        except Exception as e:
            print('Could not extract Close price:', e)
else:
    print('gme_data is empty. Add gme_stock.csv or ensure yfinance fetch worked and re-run.')

## 8. Question 7 — Tesla trend between 2018 and 2021 (2 points)

**Answer (how to derive in the notebook):**

Visually inspect the Tesla stock price and revenue plots between 2018 and 2021. Typical observations — you should draw conclusions from the actual plotted data when you run the notebook. Example expected trend:

- Tesla's stock price increased substantially between 2019 and 2021 with strong growth in market capitalization.
- Revenue showed steady year-over-year increases, reflecting growing vehicle deliveries and energy products.

(When you run the notebook, look at the plotted region 2018–2021 to produce the concrete sentence required by the assignment.)

## 9. Question 8 — GameStop key observation (2 points)

**Answer (how to derive in the notebook):**

- GameStop's stock price experienced a dramatic short squeeze event in early 2021 (large spike), while its revenue did not increase proportionally — demonstrating a decoupling of market price from company fundamentals during that event.

(Confirm by inspecting the plotted stock price and revenue series for GameStop in the notebook.)

## 10. Final: Save cleaned CSVs and results

We save cleaned versions of the data used in the notebook so they can be uploaded to your GitHub repo. This helps ensure the grader sees the same outputs.

In [ ]:
# Save cleaned outputs for reproducibility
if not tesla_data.empty:
    tesla_data.to_csv('tesla_stock_cleaned.csv', index=False)
if not tesla_revenue.empty:
    tesla_revenue.to_csv('tesla_revenue_cleaned.csv', index=False)
if not gme_data.empty:
    gme_data.to_csv('gme_stock_cleaned.csv', index=False)
if not gme_revenue.empty:
    gme_revenue.to_csv('gme_revenue_cleaned.csv', index=False)

print('Saved cleaned CSVs where available.'))
